### Import dat z excel souborů stažených z valuo do databáze
[valuo - zakoupené cenové údaje](https://profi.valuo.cz/cadastre/prices/purchased)

In [ ]:
%load_ext sql
import urllib.parse
from sqlalchemy import create_engine
import pandas as pd
import os

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = "mssql+pyodbc:///?odbc_connect=%s" % params

# Nastavení připojení
%sql $connection_url

In [ ]:
%%sql
-- select * from [dbo].[Valuo_data]

select top 10 
		cast(adresa as nvarchar(max)) as adresa
		, lat
		, lon
		, nemovitost
		, plocha
		, mena
		, cenovy_udaj
		, cenovy_udaj/plocha as jc
from valuo_data 
where lat is not null and lon is not null 
		and lat != 0 and lon != 0 
		and nemovitost = 'jednotka' 
		and cenovy_udaj is not null 
		and cenovy_udaj != 0 
		and plocha > 0
		and mena = 'czk'
group by cast(adresa as nvarchar(max)), lat, lon, nemovitost, plocha, mena, cenovy_udaj
order by jc asc

In [ ]:
import os
import pandas as pd
import urllib
import time
import numpy as np
from tqdm import tqdm
from sqlalchemy import create_engine, text
from opencage.geocoder import OpenCageGeocode    #https://opencagedata.com/dashboard#geocoding

# ======= KONFIGURACE ======= #
API_KEY = "85af71fbd7334627a5b84894066a8a18"
directory = r"C:\\Users\\ijttr\\OneDrive\\Dokumenty\\PROG\\PYTHON\\DATA_ANALYSIS\\VALUO\\data"

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření SQLAlchemy engine
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"
engine = create_engine(connection_url)

# Geokódovací API
geocoder = OpenCageGeocode(API_KEY)

# ======= MAXIMÁLNÍ DÉLKY SLOUPCŮ V DB ======= #
COLUMN_LENGTHS = {
    "cislo_vkladu": 50,
    "listina": 4000,
    "nemovitost": 50,
    "typ": 100,
    "adresa": 200,
    "mena": 10,
    "typ_plochy": 100,
    "popis": 400,
    "okres": 100,
    "kat_uzemi": 100
}

# ======= FUNKCE ======= #

def get_excel_files(directory):
    """Najde všechny Excel soubory v dané složce a podadresářích."""
    excel_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith((".xls", ".xlsx")):
                excel_files.append(os.path.join(root, file))
    if not excel_files:
        raise FileNotFoundError("❌ Žádné Excel soubory nenalezeny v dané složce!")
    return excel_files

def read_excel(file):
    """Načte data z Excelu do pandas DataFrame a přejmenuje sloupce podle mapování."""
    df = pd.read_excel(file, dtype=str)
    df.rename(columns=COLUMN_MAPPING, inplace=True)
    return df

def clean_nan_values(df):
    """Nahradí `NaN` a `None` hodnoty v povinných sloupcích a ořízne hodnoty na maximální délku."""
    df = df.replace({np.nan: None, 'nan': None, pd.NA: None})

    # ✅ Povinné textové hodnoty nahrazujeme výchozími hodnotami
    df["typ"] = df["typ"].fillna("Neznámé")  
    df["adresa"] = df["adresa"].fillna("Neznámá adresa")  
    df["plocha"] = df["plocha"].fillna("0")  
    
    # ✅ Oříznutí hodnot podle maximální délky sloupce v databázi
    for col, max_length in COLUMN_LENGTHS.items():
        if col in df.columns:
            df[col] = df[col].astype(str).str.slice(0, max_length)

    return df

def insert_to_db(df):
    """Vkládá nové záznamy do databáze s kontrolou duplicit a opravou `NaN` hodnot."""
    if df.empty:
        return 0

    df = clean_nan_values(df)  

    new_inserts = 0
    with engine.begin() as conn:
        for _, row in df.iterrows():
            result = conn.execute(text("""
                IF NOT EXISTS (
                    SELECT 1 FROM Valuo_data WHERE 
                    cislo_vkladu = :cislo_vkladu AND datum_podani = :datum_podani AND datum_zplatneni = :datum_zplatneni AND 
                    CAST(listina AS NVARCHAR(MAX)) = :listina AND nemovitost = :nemovitost AND typ = :typ AND adresa = :adresa AND 
                    cenovy_udaj = :cenovy_udaj AND mena = :mena AND ISNULL(plocha, '') = ISNULL(:plocha, '') AND 
                    ISNULL(typ_plochy, '') = ISNULL(:typ_plochy, '') AND ISNULL(popis, '') = ISNULL(:popis, '') AND 
                    okres = :okres AND kat_uzemi = :kat_uzemi AND rok = :rok AND mesic = :mesic
                ) 
                BEGIN
                    INSERT INTO Valuo_data (cislo_vkladu, datum_podani, datum_zplatneni, listina, nemovitost, typ, adresa, cenovy_udaj, 
                        mena, plocha, typ_plochy, popis, okres, kat_uzemi, rok, mesic) 
                    VALUES (:cislo_vkladu, :datum_podani, :datum_zplatneni, :listina, :nemovitost, :typ, :adresa, :cenovy_udaj, 
                        :mena, :plocha, :typ_plochy, :popis, :okres, :kat_uzemi, :rok, :mesic);
                END
            """), row.to_dict())

            if result.rowcount > 0:
                new_inserts += 1

    return new_inserts

# ======= HLAVNÍ SPUŠTĚNÍ ======= #

if __name__ == "__main__":
    try:
        excel_files = get_excel_files(directory)  
        total_files_processed = 0
        total_records = 0
        newly_inserted_records = 0

        for file in tqdm(excel_files, desc="📂 Zpracování souborů"):
            df = read_excel(file)
            total_records += len(df)
            new_records = insert_to_db(df)
            newly_inserted_records += new_records
            total_files_processed += 1

        with engine.connect() as conn:
            missing_before = conn.execute(text("SELECT COUNT(*) FROM Valuo_data WHERE LAT IS NULL AND LON IS NULL AND adresa IS NOT NULL")).scalar()

        updated_rows, api_calls = fetch_missing_coordinates()

        with engine.connect() as conn:
            missing_after = conn.execute(text("SELECT COUNT(*) FROM Valuo_data WHERE LAT IS NULL AND LON IS NULL AND adresa IS NOT NULL")).scalar()

        print(f"\n📊 Statistiky zpracování:")
        print(f"📁 Celkem souborů: {total_files_processed}")
        print(f"📝 Celkem záznamů: {total_records}")
        print(f"✅ Skutečně nově vložené záznamy: {newly_inserted_records}")
        print(f"📍 Nově stažené GPS souřadnice: {updated_rows}")
        print(f"🛰️ API dotazů na GPS: {api_calls}")
        print(f"⏳ Záznamy stále bez souřadnic: {missing_after}")

    except FileNotFoundError as e:
        print(e)


In [65]:
import os
import pandas as pd
import urllib
import time
import numpy as np
from tqdm import tqdm
from sqlalchemy import create_engine, text
from opencage.geocoder import OpenCageGeocode

# ======= KONFIGURACE ======= #
API_KEY = "85af71fbd7334627a5b84894066a8a18"
directory = r"C:\\Users\\ijttr\\OneDrive\\Dokumenty\\PROG\\PYTHON\\DATA_ANALYSIS\\VALUO\\data"

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření SQLAlchemy engine
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"
engine = create_engine(connection_url)

# Geokódovací API
geocoder = OpenCageGeocode(API_KEY)

# ======= MAXIMÁLNÍ DÉLKY SLOUPCŮ V DB ======= #
COLUMN_LENGTHS = {
    "cislo_vkladu": 50,
    "listina": 4000,
    "nemovitost": 50,
    "typ": 100,
    "adresa": 200,
    "mena": 10,
    "typ_plochy": 100,
    "popis": 400,
    "okres": 100,
    "kat_uzemi": 100
}

# ======= FUNKCE ======= #

def get_excel_files(directory):
    """Najde všechny Excel soubory v dané složce a podadresářích."""
    excel_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith((".xls", ".xlsx")):
                excel_files.append(os.path.join(root, file))
    if not excel_files:
        raise FileNotFoundError("❌ Žádné Excel soubory nenalezeny v dané složce!")
    return excel_files

def read_excel(file):
    """Načte data z Excelu do pandas DataFrame a přejmenuje sloupce podle mapování."""
    df = pd.read_excel(file, dtype=str)
    df.rename(columns=COLUMN_MAPPING, inplace=True)
    return df

def clean_nan_values(df):
    """Nahradí `NaN` a `None` hodnoty v povinných sloupcích a ořízne hodnoty na maximální délku."""
    df = df.replace({np.nan: None, 'nan': None, pd.NA: None})

    # ✅ Povinné textové hodnoty nahrazujeme výchozími hodnotami
    df["typ"] = df["typ"].fillna("Neznámé")  
    df["adresa"] = df["adresa"].fillna("Neznámá adresa")  
    df["plocha"] = df["plocha"].fillna("0")  
    
    # ✅ Oříznutí hodnot podle maximální délky sloupce v databázi
    for col, max_length in COLUMN_LENGTHS.items():
        if col in df.columns:
            df[col] = df[col].astype(str).str.slice(0, max_length)

    return df

def insert_to_db(df):
    """Vkládá nové záznamy do databáze s kontrolou duplicit a opravou `NaN` hodnot."""
    if df.empty:
        return 0

    df = clean_nan_values(df)  

    new_inserts = 0
    with engine.begin() as conn:
        for _, row in df.iterrows():
            result = conn.execute(text("""
                IF NOT EXISTS (
                    SELECT 1 FROM Valuo_data WHERE 
                    cislo_vkladu = :cislo_vkladu AND datum_podani = :datum_podani AND datum_zplatneni = :datum_zplatneni AND 
                    CAST(listina AS NVARCHAR(MAX)) = :listina AND nemovitost = :nemovitost AND typ = :typ AND adresa = :adresa AND 
                    cenovy_udaj = :cenovy_udaj AND mena = :mena AND ISNULL(plocha, '') = ISNULL(:plocha, '') AND 
                    ISNULL(typ_plochy, '') = ISNULL(:typ_plochy, '') AND ISNULL(popis, '') = ISNULL(:popis, '') AND 
                    okres = :okres AND kat_uzemi = :kat_uzemi AND rok = :rok AND mesic = :mesic
                ) 
                BEGIN
                    INSERT INTO Valuo_data (cislo_vkladu, datum_podani, datum_zplatneni, listina, nemovitost, typ, adresa, cenovy_udaj, 
                        mena, plocha, typ_plochy, popis, okres, kat_uzemi, rok, mesic) 
                    VALUES (:cislo_vkladu, :datum_podani, :datum_zplatneni, :listina, :nemovitost, :typ, :adresa, :cenovy_udaj, 
                        :mena, :plocha, :typ_plochy, :popis, :okres, :kat_uzemi, :rok, :mesic);
                END
            """), row.to_dict())

            if result.rowcount > 0:
                new_inserts += 1

    return new_inserts

def fetch_missing_coordinates():
    """Doplňuje GPS souřadnice pro záznamy v databázi, které nemají LAT a LON, s limitem 2500 API dotazů/den."""

    with engine.connect() as conn:
        rows = conn.execute(text("""
            SELECT adresa, LAT, LON FROM Valuo_data WHERE LAT IS NOT NULL AND LON IS NOT NULL
        """)).fetchall()

    existing_coords = {adresa: (LAT, LON) for adresa, LAT, LON in rows}

    with engine.connect() as conn:
        rows = conn.execute(text("""
            SELECT id, adresa FROM Valuo_data WHERE LAT IS NULL AND LON IS NULL AND adresa IS NOT NULL
        """)).fetchall()

    updated_rows = 0
    api_calls = 0
    copied_from_db = 0  
    skipped_due_to_api_limit = 0  
    api_limit_reached = False  

    progress_bar = tqdm(rows, desc="🌍 Stahování GPS souřadnic", unit="záznam", dynamic_ncols=True)

    for row in progress_bar:
        record_id, address = row
        lat, lon, gps_status = None, None, "NULL"  

        if address in existing_coords:
            lat, lon = existing_coords[address]  
            gps_status = "OK"
            copied_from_db += 1  
            status_text = f"📍 Kopíruji z DB: {copied_from_db} | 🌍 API: {api_calls} | 🚫 Přeskočeno kvůli API: {skipped_due_to_api_limit}"
        else:
            if api_limit_reached:
                skipped_due_to_api_limit += 1
                status_text = f"📍 Kopíruji z DB: {copied_from_db} | 🌍 API: {api_calls} | 🚫 Přeskočeno kvůli API: {skipped_due_to_api_limit}"
                progress_bar.set_postfix_str(status_text)
                continue  

            time.sleep(1)
            try:
                result = geocoder.geocode(address)
                if result:
                    lat = round(result[0]['geometry']['lat'], 7)  
                    lon = round(result[0]['geometry']['lng'], 7)  
                    api_calls += 1  

                    if 48.55 <= lat <= 51.06 and 12.09 <= lon <= 18.86:
                        existing_coords[address] = (lat, lon)
                        gps_status = "OK"
                    else:
                        lat, lon = None, None  
                        gps_status = "ERR"

                else:
                    lat, lon = None, None  
                    gps_status = "ERR"

            except Exception as e:
                if "RateLimitExceededError" in str(e):  
                    api_limit_reached = True
                    print("\n🚨 DENNÍ LIMIT API (2500 dotazů) BYL DOSAŽEN! 🚨")
                    skipped_due_to_api_limit += 1
                    continue  

        with engine.begin() as conn:
            conn.execute(text("""
                UPDATE Valuo_data 
                SET LAT=:lat, LON=:lon, GPS_API_info=:gps_status
                WHERE id=:id
            """), {"lat": lat, "lon": lon, "gps_status": gps_status, "id": record_id})
            updated_rows += 1

        status_text = f"📍 Kopíruji z DB: {copied_from_db} | 🌍 API: {api_calls} | 🚫 Přeskočeno kvůli API: {skipped_due_to_api_limit}"
        progress_bar.set_postfix_str(status_text)

    return updated_rows, api_calls, copied_from_db






# ======= HLAVNÍ SPUŠTĚNÍ ======= #

if __name__ == "__main__":
    try:
        excel_files = get_excel_files(directory)  
        total_files_processed = 0
        total_records = 0
        newly_inserted_records = 0

        for file in tqdm(excel_files, desc="📂 Zpracování souborů"):
            df = read_excel(file)
            total_records += len(df)
            new_records = insert_to_db(df)
            newly_inserted_records += new_records
            total_files_processed += 1

        updated_rows, api_calls, copied_from_db = fetch_missing_coordinates()

        print(f"\n📊 Statistiky zpracování:")
        print(f"📁 Celkem souborů: {total_files_processed}")
        print(f"📝 Celkem záznamů: {total_records}")
        print(f"✅ Skutečně nově vložené záznamy: {newly_inserted_records}")
        print(f"📍 Zkopírované GPS z DB: {copied_from_db}")  # ✅ Nová statistika
        print(f"🌍 Nově stažené GPS přes API: {api_calls}")  # ✅ Nová statistika
        print(f"🛰️ API dotazů na GPS: {api_calls}")  # Už bylo, ale nechávám
        print(f"⏳ Záznamy stále bez souřadnic: {missing_after}")

    except FileNotFoundError as e:
        print(e)


❌ Žádné Excel soubory nenalezeny v dané složce!
